In [197]:
import pandas as pd
import math
df1=pd.read_excel(r'C:/Users/kantw/Downloads/Data2.xlsx')
df=pd.read_excel(r'C:/Users/kantw/Downloads/Data.xlsx')

In [259]:
df

,frame,id,x (long. position),y (lateral position),length,width,laneId
0,1,1,366.39,32.17,5.15,1.82,6
1,2,1,366.98,32.17,5.15,1.82,6
2,3,1,367.61,32.18,5.15,1.82,6
3,4,1,368.25,32.19,5.15,1.82,6
4,5,1,368.90,32.20,5.15,1.82,6
...,...,...,...,...,...,...,...
344292,15628,1478,407.35,35.65,6.97,2.43,7
344293,15629,1478,408.52,35.65,6.97,2.43,7
344294,15630,1478,409.69,35.65,6.97,2.43,7
344295,15631,1478,410.86,35.65,6.97,2.43,7


In [260]:
df1

,vehicle Id,length,width,Type
0,1,5.15,1.82,Car
1,4,4.95,1.92,Car
2,5,5.56,1.92,Car
3,6,6.47,2.02,Truck
4,7,4.95,1.72,Car
...,...,...,...,...
1613,2843,8.49,2.50,Truck
1614,2844,4.85,2.02,Car
1615,2845,3.23,1.82,Car
1616,2849,5.86,1.82,Car


In [283]:
def getAnswers(df, df1):
    final_answers = {}
    
    # get length and width for every vehicle id
    l_w_data = {}
    df1_ids = list(df1['vehicle Id'])
    df1_length = list(df1['length'])
    df1_width = list(df1['width '])
    for i in range(len(df1_ids)):
        l_w_data[df1_ids[i]] = (df1_length[i], df1_width[i])

    #get all frames for a given id
    df_ids = list(df['id'])
    df_frames = list(df['frame'])
    total_frame = {}
    for i in range(len(df_ids)):
        if df_ids[i] in total_frame:
            total_frame[df_ids[i]].append(df_frames[i])
        else:
            total_frame[df_ids[i]] = [df_frames[i]]

    lst = list(df['y (lateral position)'])
    lst_x = list(df['x (long. position)'])

    all_p = []
    r_l = []
    for i in range(len(lst_x) - 1):
        if lst_x[i] <= lst_x[i+1]:
            r_l.append((lst_x[i], lst[i]))
        else:
            r_l.append((lst_x[i], lst[i]))
            all_p.append(r_l)
            r_l = []
    all_p.append(r_l)

    frame_l = list(df['frame'])
    lane_l = list(df['laneId'])
    id_l = list(df['id'])

    main_d = {}
    for i in range(len(frame_l)):
        f = frame_l[i]
        s = lane_l[i]
        x = lst_x[i]
        r_id = id_l[i]
        if f in main_d:
            if s in main_d[f]:
                main_d[f][s].append((r_id, x))
            else:
                main_d[f][s] = [(r_id, x)]
        else:
            main_d[f] = {s: [(r_id, x)]}

    all_l_avg = {6:[], 7: [], 8: []}
    for i in main_d:
        for j in main_d[i]:
            r_lst = main_d[i][j]
            r_diff = []
            for k in range(len(r_lst)-1):
                r_diff.append(r_lst[k][1] - r_lst[k+1][1] - l_w_data[r_lst[k+1][0]][0])
            if len(r_diff) != 0:
                all_l_avg[j].append(sum(r_diff)/len(r_diff))
    all_l_avg

    s_lst = all_l_avg[6]
    sixth = 0
    if len(s_lst) != 0:
        sixth = round(sum(s_lst)/len(s_lst),2)
    

    s_lst = all_l_avg[7]
    seventh = 0
    if len(s_lst) != 0:
        seventh = round(sum(s_lst)/len(s_lst),2)
    

    s_lst = all_l_avg[8]
    eight = 0
    if len(s_lst) != 0:
        eight = round(sum(s_lst)/len(s_lst),2)
    

    #Space
    final_answers[1] = ("Space Headway", round((sixth+seventh+eight)/3, 2), "m/veh")

    #Time Headway
    l_frame = []
    for i in total_frame:
        l_frame.append(max(total_frame[i]))         
    l_frame.sort()
    t_h_lst = []
    for i in range(len(all_p) - 1):
        t_h_lst.append((l_frame[i+1] - l_frame[i])/25)
    t_h_lst
    final_answers[2] = ("Time Headway", round(sum(t_h_lst)/len(t_h_lst), 2), "s/veh")


    d_lst = []
    for i in main_d:
        cnt = 0
        for j in main_d[i]:
            cnt += len(main_d[i][j])
        d_lst.append(cnt/0.42)
    d_lst

    #Density
    final_answers[3] = ("Density", round(sum(d_lst)/len(d_lst), 2), "veh/km")


    f_lst = {}
    for i in l_frame:
        key = math.ceil(i/(25*1*60))
        if key in f_lst:
            f_lst[key].append(i)
        else:
            f_lst[key] = [i]
    f_lst

    #avg
    avg_lst = []
    for i in f_lst:
        avg_lst.append(len(f_lst[i]))
    #Flow
    final_answers[4] = ("Flow", round((sum(avg_lst)/len(avg_lst))*60, 2), "veh/hr")

    #avg_speed
    avg_speed = []
    for i in total_frame:
        avg_speed.append((0.42*25*3600)/len(total_frame[i])) 
    avg_speed

    # km/h Velocity
    final_answers[5] = ("Velocity", round(sum(avg_speed)/len(avg_speed), 2), "km/hr")
    
    return final_answers

## First Question

In [266]:
first = getAnswers(df, df1)

In [267]:
m_f = first
first

{1: ('Space Headway', 51.16, 'm/veh'),
 2: ('Time Headway', 0.77, 's/veh'),
 3: ('Density', 52.44, 'veh/km'),
 4: ('Flow', 4423.64, 'veh/hr'),
 5: ('Velocity', 93.02, 'km/hr')}

## Q = K*V

In [268]:
print("Q: "+str(first[4][1])+", K*V: "+str(round(first[3][1] * first[5][1], 2)))

Q: 4423.64, K*V: 4877.97


## % Error

In [269]:
((4877.97 - 4423.64)/4877.97)*100

9.313915419734027

## Second

In [270]:
df2 = df1[df1.Type == "Car"]
df2

,vehicle Id,length,width,Type
0,1,5.15,1.82,Car
1,4,4.95,1.92,Car
2,5,5.56,1.92,Car
4,7,4.95,1.72,Car
5,8,4.45,1.72,Car
...,...,...,...,...
1612,2841,4.75,1.82,Car
1614,2844,4.85,2.02,Car
1615,2845,3.23,1.82,Car
1616,2849,5.86,1.82,Car


In [271]:
lst_ids = list(df2['vehicle Id'])
df3 = df[df.id.isin(lst_ids)]
df3

,frame,id,x (long. position),y (lateral position),length,width,laneId
0,1,1,366.39,32.17,5.15,1.82,6
1,2,1,366.98,32.17,5.15,1.82,6
2,3,1,367.61,32.18,5.15,1.82,6
3,4,1,368.25,32.19,5.15,1.82,6
4,5,1,368.90,32.20,5.15,1.82,6
...,...,...,...,...,...,...,...
343925,15599,1476,408.81,30.84,4.75,1.72,6
343926,15600,1476,410.05,30.84,4.75,1.72,6
343927,15601,1476,411.28,30.85,4.75,1.72,6
343928,15602,1476,412.52,30.85,4.75,1.72,6


## Car

In [272]:
second_car = getAnswers(df3, df2)

In [273]:
second_car

{1: ('Space Headway', 66.16, 'm/veh'),
 2: ('Time Headway', 0.87, 's/veh'),
 3: ('Density', 45.55, 'veh/km'),
 4: ('Flow', 3889.09, 'veh/hr'),
 5: ('Velocity', 94.34, 'km/hr')}

## Q = K*V

In [274]:
first = second_car
print("Q: "+str(first[4][1])+", K*V: "+str(round(first[3][1] * first[5][1], 2)))

Q: 3889.09, K*V: 4297.19


## % Error

In [275]:
((4297.19 - 3889.09)/4297.19)*100

9.496903790616647

## Truck

In [276]:
df4 = df1[df1.Type == "Truck"]
df4

,vehicle Id,length,width,Type
3,6,6.47,2.02,Truck
6,10,6.57,2.12,Truck
14,23,18.80,2.50,Truck
25,45,7.38,2.50,Truck
45,88,8.39,2.50,Truck
...,...,...,...,...
1599,2816,16.78,2.50,Truck
1602,2820,16.88,2.50,Truck
1605,2826,17.08,2.50,Truck
1610,2836,17.08,2.50,Truck


In [277]:
lst_ids = list(df4['vehicle Id'])
df5 = df[df.id.isin(lst_ids)]
df5

,frame,id,x (long. position),y (lateral position),length,width,laneId
310,1,6,300.31,35.78,6.47,2.02,7
311,2,6,300.90,35.77,6.47,2.02,7
312,3,6,301.51,35.77,6.47,2.02,7
313,4,6,302.14,35.76,6.47,2.02,7
314,5,6,302.78,35.75,6.47,2.02,7
...,...,...,...,...,...,...,...
344292,15628,1478,407.35,35.65,6.97,2.43,7
344293,15629,1478,408.52,35.65,6.97,2.43,7
344294,15630,1478,409.69,35.65,6.97,2.43,7
344295,15631,1478,410.86,35.65,6.97,2.43,7


In [278]:
second_truck = getAnswers(df5, df4)

In [279]:
second_truck

{1: ('Space Headway', 73.19, 'm/veh'),
 2: ('Time Headway', 6.38, 's/veh'),
 3: ('Density', 7.46, 'veh/km'),
 4: ('Flow', 534.55, 'veh/hr'),
 5: ('Velocity', 83.41, 'km/hr')}

## Q = KV

In [280]:
first = second_truck
print("Q: "+str(first[4][1])+", K*V: "+str(round(first[3][1] * first[5][1], 2)))

Q: 534.55, K*V: 622.24


## % Error

In [281]:
((622.24 - 534.55)/622.24)*100

14.092633067626647

## Third

In [282]:
print("Capacity: "+ str(math.ceil(1000*m_f[5][1] / (m_f[1][1]*3))) + " veh/hr/lane")

Capacity: 607 veh/hr/lane


## Assumptions

### 1. Video taken at 25 FPS
### 2. Coordinates assume to be of a car's rear bumper
### 3. The last frame of every car is at the same ordinate.